In [3]:
!pip install py2neo

In [4]:
from py2neo import Graph

In [5]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [6]:
type(graph)

py2neo.database.Graph

In [134]:
import pandas as pd
import numpy as np
df_empl = pd.read_csv("employees.csv") #employee info
df_act = pd.read_csv("actors.csv") #actor info
df_hobby = pd.read_csv("hobby.csv") #character hobby info
df_hob_pop = pd.read_csv("hobby_popularity.csv") #hobby popularity
df_jobs = pd.read_csv("jobs.csv") #character dunder mif job

df_jobs.drop([2,11], axis = 0, inplace = True) # drop unnecessary rows
df_jobs["job"] = df_jobs["job"].replace(['HR'], 'human resources')
df_act.drop([15], axis = 0, inplace = True)
df_hobby.drop([13, 14], axis = 0, inplace = True)
df_hob_pop.drop([6], axis = 0, inplace = True)

In [135]:
graph.run("MATCH (n) DETACH DELETE n")
from py2neo import Node
list = []

# populate db with all office character nodes
for i in range(0,len(df_empl)):
    person = Node("Person", name = df_empl['Employee_Name'].iloc[i], branch = df_empl['last_branch'].iloc[i], episodes = (df_empl['episodes'].iloc[i]).item(), years = (df_empl['years_at_office'].iloc[i]).item(), main_character = (df_empl["is_main_character"].iloc[i]).item())
    graph.create(person)
    
# populate db with all office jobs nodes 
for i in range(0, len(df_jobs)):
    job = Node("Job", name = (df_jobs['job'].iloc[i]).title(), salary = df_jobs['salary'].iloc[i], education = df_jobs['typical educational requirement'].iloc[i])
    graph.create(job)

#populate db with all actor nodes
for i in range(0, len(df_act)):
    actor = Node("Actor", name = df_act['name'].iloc[i], age = (df_act['age'].iloc[i]).item(), net_worth = (df_act['net_worth'].iloc[i]).item(), emmy_nominations = (df_act['emmy_nominations'].iloc[i]).item())
    graph.create(actor)

#populate db with all hobby nodes
for i in range(0, len(df_hobby)):
    hobby = Node("Hobby", hobby = (df_hobby['hobby'].iloc[i]).title(), cost_level = df_hobby['cost_level'].iloc[i])
    graph.create(hobby)
    
#populate db with all hobby cat nodes
for i in range(0, len(df_hob_pop)):
    hob_cat = Node("Hobby Category", hobby_category = (df_hob_pop['hobby_category'].iloc[i]).title(), percentage = (df_hob_pop['pop_by_percentage'].iloc[i]).item())
    graph.create(hob_cat)

#social group nodes    
fin_club = Node("Social Groups", name = "Finer Thing Club")
ppp_c = Node("Social Groups", name = "Party Planning Committee")
kev_z = Node("Social Groups", name = "Kevin & the Zits")


In [136]:
from py2neo import Relationship
WorksAs = Relationship.type("WORKS_AS")
ActsAs = Relationship.type("ACTS_AS")
HobbyIs = Relationship.type("HOBBY_IS")
HobbyCategoryIs = Relationship.type("HOBBY_CATEGORY_IS")
Frenemy = Relationship.type("FRENEMIES")
Dated = Relationship.type("DATED")
Friends = Relationship.type("FRIENDS_WITH")
Enemy = Relationship.type("ENEMIES")
Assitant = Relationship.type("ASSISTANT_TO_REGIONAL_MANAGER")
Married = Relationship.type("MARRIED_TO")
IsIn = Relationship.type("IS_IN")
NonPermanentM = Relationship.type("HONORARY_MEMBER")
Rejected = Relationship.type("REJECTED")

from py2neo import Graph, NodeMatcher
#graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)


for i in range(0,len(df_empl)): #matching character with jobs
    empl_job = df_jobs[df_jobs['id'] == df_empl["job_ID"].iloc[i].item()][["job"]].iloc[0][0].title()
    per = df_empl["Employee_Name"].iloc[i]
    per_node = matcher.match("Person").where(name=per).first()
    empl_node = matcher.match("Job").where(name=empl_job).first()
    graph.create(WorksAs(per_node, empl_node))
    
#matching character with actor
for i in range(0,len(df_empl)):
    empl_act = df_act[df_act['id'] == df_empl["actor_ID"].iloc[i].item()][["name"]].iloc[0][0].title()
    per = df_empl["Employee_Name"].iloc[i]
    per_node = matcher.match("Person").where(name=per).first()
    act_node = matcher.match("Actor").where(name=empl_act).first()
    graph.create(ActsAs(act_node, per_node))

#matching character with hobby
for i in range(0,len(df_empl)): 
    empl_hob = df_hobby[df_hobby['id'] == df_empl["hobby_ID"].iloc[i].item()][["hobby"]].iloc[0][0].title()
    per = df_empl["Employee_Name"].iloc[i]
    per_node = matcher.match("Person").where(name=per).first()
    hobby_node = matcher.match("Hobby").where(hobby=empl_hob).first()
    graph.create(HobbyIs(per_node, hobby_node))

#matching hobby with hobby category
for i in range(0,len(df_hobby)-1): 
    hob_c = df_hob_pop[df_hob_pop['hobby_cat_id'] == df_hobby["hobby_cat_id"].iloc[i].item()][["hobby_category"]].iloc[0][0].title()
    hobby_b = (df_hobby["hobby"].iloc[i]).title()
    hobbyc_node = matcher.match("Hobby Category").where(hobby_category=hob_c).first()
    hobby_node = matcher.match("Hobby").where(hobby=hobby_b).first()
    graph.create(HobbyCategoryIs(hobby_node, hobbyc_node))
    
#relationships
JIM = matcher.match("Person").where(name="Jim Halpert").first()
DWIGHT = matcher.match("Person").where(name="Dwight Schrute").first()
KELLY = matcher.match("Person").where(name="Kelly Kapoor").first()
RYAN = matcher.match("Person").where(name="Ryan Howard").first()
DARRYL = matcher.match("Person").where(name="Darryl Philbin").first()
MICHAEL = matcher.match("Person").where(name="Michael Scott").first()
JAN = matcher.match("Person").where(name="Jan Levinson").first()
ANGELA = matcher.match("Person").where(name="Angela Martin").first()
ANDY = matcher.match("Person").where(name="Andy Bernard").first()
ERIN = matcher.match("Person").where(name="Erin Hannon").first()
PAM = matcher.match("Person").where(name="Pam Beesly").first()
OSCAR = matcher.match("Person").where(name="Oscar Martinez").first()
TOBY = matcher.match("Person").where(name="Toby Flenderson").first()
KEVIN = matcher.match("Person").where(name="Kevin Malone").first()
PHYLLIS = matcher.match("Person").where(name="Phyllis Vance").first()

graph.create(Frenemy(JIM, DWIGHT))
graph.create(Frenemy(DWIGHT, JIM))

graph.create(Dated(KELLY, RYAN))
graph.create(Dated(RYAN, KELLY))
graph.create(Dated(KELLY, DARRYL))
graph.create(Dated(DARRYL, KELLY))
graph.create(Dated(MICHAEL, JAN))
graph.create(Dated(JAN, MICHAEL))
graph.create(Dated(ANGELA, ANDY))
graph.create(Dated(ANDY, ANGELA))
graph.create(Dated(ANDY, ERIN))
graph.create(Dated(ERIN, ANDY))

graph.create(Friends(MICHAEL, PAM))
graph.create(Friends(PAM, MICHAEL))

graph.create(Enemy(PAM, ANGELA))
graph.create(Enemy(ANGELA, PAM))

graph.create(Assitant(DWIGHT, MICHAEL))

graph.create(Married(DWIGHT, ANGELA))
graph.create(Married(ANGELA, DWIGHT))
graph.create(Married(PAM, JIM))
graph.create(Married(JIM, PAM))

#matching character with social group
graph.create(IsIn(PAM, fin_club))
graph.create(IsIn(OSCAR, fin_club))
graph.create(IsIn(TOBY, fin_club))
graph.create(Rejected(fin_club, ANDY))
graph.create(NonPermanentM(JIM, fin_club))

graph.create(IsIn(PAM, ppp_c))
graph.create(IsIn(PHYLLIS, ppp_c))
graph.create(IsIn(ANGELA, ppp_c))
graph.create(NonPermanentM(KELLY, ppp_c))
graph.create(NonPermanentM(RYAN, ppp_c))
graph.create(NonPermanentM(OSCAR, ppp_c))

graph.create(IsIn(KEVIN, kev_z))
graph.create(IsIn(ANDY, kev_z))
graph.create(IsIn(DARRYL, kev_z))
